In [3]:
!pip install shapely

In [5]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import healpy as hp

from rubin_sim.data import get_baseline
import sqlite3


# import part1.py

In [6]:
# find the baseline survey simulation file that got downloaded with rubin_sim
baseline_file = '/Users/apple/rubin_sim_data/sim_baseline/baseline.db'

# Conenct to the sqlite database
con = sqlite3.connect(baseline_file)

# We can just load the whole thing into a dataframe
df = pd.read_sql('select * from observations;', con)
con.close()

In [7]:
# take a look at what the data looks like
df

,observationId,fieldRA,fieldDec,observationStartMJD,flush_by_mjd,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,...,sunAz,sunRA,sunDec,moonRA,moonDec,moonDistance,solarElong,moonPhase,cummTelAz,scripted_id
0,0,310.024480,-60.812928,60218.001806,60218.023576,30.0,y,-297.249225,-297.249225,2,...,255.593220,186.644048,-2.870827,27.609463,11.956111,94.490314,102.958651,87.407902,169.454444,0
1,1,310.601871,-63.561425,60218.002254,60218.023576,30.0,y,-297.708278,-297.708278,2,...,255.500445,186.644453,-2.871001,27.615338,11.959438,95.029204,101.743959,87.404494,170.502875,0
2,2,311.292611,-66.317774,60218.002703,60218.023576,30.0,y,-297.909620,-297.909620,2,...,255.407493,186.644858,-2.871176,27.621208,11.962765,95.563446,100.497860,87.401088,171.406738,0
3,3,312.140731,-69.082666,60218.003152,60218.023576,30.0,y,-297.838337,-297.838337,2,...,255.314364,186.645264,-2.871350,27.627073,11.966093,96.092842,99.221261,87.397685,172.197791,0
4,4,304.170163,-73.375442,60218.003623,60218.023576,30.0,y,-309.290623,-309.290623,2,...,255.216260,186.645690,-2.871533,27.633232,11.969593,99.605370,94.821142,87.394110,177.239803,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2081744,2081744,347.124408,-44.121665,63870.104312,63870.135744,30.0,u,16.313879,16.313879,1,...,226.952534,186.231318,-2.693059,256.910781,-17.679803,77.943326,130.154875,39.323876,118.256160,0
2081745,2081745,344.049134,-45.895531,63870.104746,63870.135744,30.0,u,5.751050,5.751050,1,...,226.779699,186.231710,-2.693228,256.915974,-17.679424,75.449290,127.589799,39.326371,127.956145,0
2081746,2081746,340.774233,-47.593274,63870.105180,63870.135744,30.0,u,-4.958170,-4.958170,1,...,226.606027,186.232103,-2.693397,256.921178,-17.679043,72.960208,124.997243,39.328872,136.957654,0
2081747,2081747,337.288929,-49.202077,63870.105614,63870.135744,30.0,u,-15.452126,-15.452126,1,...,226.432028,186.232495,-2.693565,256.926379,-17.678664,70.481515,122.382915,39.331373,144.924885,0


In [8]:
df.columns

Index(['observationId', 'fieldRA', 'fieldDec', 'observationStartMJD',
       'flush_by_mjd', 'visitExposureTime', 'filter', 'rotSkyPos',
       'rotSkyPos_desired', 'numExposures', 'airmass', 'seeingFwhm500',
       'seeingFwhmEff', 'seeingFwhmGeom', 'skyBrightness', 'night', 'slewTime',
       'visitTime', 'slewDistance', 'fiveSigmaDepth', 'altitude', 'azimuth',
       'paraAngle', 'cloud', 'moonAlt', 'sunAlt', 'note', 'fieldId',
       'proposalId', 'block_id', 'observationStartLST', 'rotTelPos',
       'rotTelPos_backup', 'moonAz', 'sunAz', 'sunRA', 'sunDec', 'moonRA',
       'moonDec', 'moonDistance', 'solarElong', 'moonPhase', 'cummTelAz',
       'scripted_id'],
      dtype='object')

***random helper functions because it wont import properly


In [8]:
from astropy import constants as const
from astropy import units as u

def satellite_mean_motion(altitude, mu=const.GM_earth, r_earth=const.R_earth):
    '''
    Compute mean motion of satellite at altitude in Earth's gravitational field.

    See https://en.wikipedia.org/wiki/Mean_motion#Formulae
    '''
    no = np.sqrt(4.0 * np.pi ** 2 * (altitude + r_earth) ** 3 / mu).to(u.day)
    return 1 / no

def tle_from_orbital_parameters(sat_name, sat_nr, epoch, inclination, raan,
                                mean_anomaly, mean_motion):
    '''
    Generate TLE strings from orbital parameters.

    Note: epoch has a very strange format: first two digits are the year, next three
    digits are the day from beginning of year, then fraction of a day is given, e.g.
    20180.25 would be 2020, day 180, 6 hours (UT?)
    '''

    # Note: RAAN = right ascention (or longitude) of ascending node

    def checksum(line):
        s = 0
        for c in line[:-1]:
            if c.isdigit():
                s += int(c)
            if c == "-":
                s += 1
        return '{:s}{:1d}'.format(line[:-1], s % 10)

    tle0 = sat_name
    tle1 = checksum(
        '1 {:05d}U 20001A   {:14.8f}  .00000000  00000-0  50000-4 '
        '0    0X'.format(sat_nr, epoch))
    tle2 = checksum(
        '2 {:05d} {:8.4f} {:8.4f} 0001000   0.0000 {:8.4f} '
        '{:11.8f}    0X'.format(
            sat_nr, inclination.to_value(u.deg), raan.to_value(u.deg),
            mean_anomaly.to_value(u.deg), mean_motion.to_value(1 / u.day)
        ))

    return '\n'.join([tle0, tle1, tle2])

def create_constellation(altitudes, inclinations, nplanes, sats_per_plane, epoch=22050.1, name='Test'):

    my_sat_tles = []
    sat_nr = 8000
    for alt, inc, n, s in zip(
            altitudes, inclinations, nplanes, sats_per_plane):

        if s == 1:
            # random placement for lower orbits
            mas = np.random.uniform(0, 360, n) * u.deg
            raans = np.random.uniform(0, 360, n) * u.deg
        else:
            mas = np.linspace(0.0, 360.0, s, endpoint=False) * u.deg
            mas += np.random.uniform(0, 360, 1) * u.deg
            raans = np.linspace(0.0, 360.0, n, endpoint=False) * u.deg
            mas, raans = np.meshgrid(mas, raans)
            mas, raans = mas.flatten(), raans.flatten()

        mm = satellite_mean_motion(alt)
        for ma, raan in zip(mas, raans):
            my_sat_tles.append(
                tle_from_orbital_parameters(
                    name+' {:d}'.format(sat_nr), sat_nr, epoch,
                    inc, raan, ma, mm))
            sat_nr += 1

    return my_sat_tles

def starlink_constellation(supersize=False, fivek=False):
    """
    Create a list of satellite TLE's
    """
    altitudes = np.array([550, 1110, 1130, 1275, 1325, 345.6, 340.8, 335.9])
    inclinations = np.array([53.0, 53.8, 74.0, 81.0, 70.0, 53.0, 48.0, 42.0])
    nplanes = np.array([72, 32, 8, 5, 6, 2547, 2478, 2493])
    sats_per_plane = np.array([22, 50, 50, 75, 75, 1, 1, 1])

    if supersize:
        # Let's make 4 more altitude and inclinations
        new_altitudes = []
        new_inclinations = []
        new_nplanes = []
        new_sat_pp = []
        for i in np.arange(0, 4):
            new_altitudes.append(altitudes+i*20)
            new_inclinations.append(inclinations+3*i)
            new_nplanes.append(nplanes)
            new_sat_pp.append(sats_per_plane)

        altitudes = np.concatenate(new_altitudes)
        inclinations = np.concatenate(new_inclinations)
        nplanes = np.concatenate(new_nplanes)
        sats_per_plane = np.concatenate(new_sat_pp)

    altitudes = altitudes * u.km
    inclinations = inclinations * u.deg
    my_sat_tles = create_constellation(altitudes, inclinations, nplanes, sats_per_plane, name='Starl')

    if fivek:
        stride = round(len(my_sat_tles)/5000)
        my_sat_tles = my_sat_tles[::stride]

    return my_sat_tles


In [9]:
import numpy as np 

def bear(latA, lonA, latB, lonB):
    """All radians
    """
    # BEAR Finds the bearing from one lat / lon point to another.
    #bearing: The horizontal angle between the astronomic meridian and a line on the Earth
    #the angle between 2 points on the sky. northpole is always at 0. How far to you have to rotate 
    result = np.arctan2(np.sin(lonB - lonA) * np.cos(latB),
                        np.cos(latA) * np.sin(latB) - np.sin(latA) * np.cos(latB) * np.cos(lonB - lonA)
                        )

    return result

def _angularSeparation(long1, lat1, long2, lat2):
    """
    angle between 2 points
    """
    ## haversine distance 
    #how far apart two points on the sky are 
    t1 = np.sin(lat2/2.0 - lat1/2.0)**2
    t2 = np.cos(lat1)*np.cos(lat2)*np.sin(long2/2.0 - long1/2.0)**2
    _sum = t1 + t2

    if np.size(_sum) == 1:
        if _sum < 0.0:
            _sum = 0.0
    else:
        _sum = np.where(_sum < 0.0, 0.0, _sum)

    return 2.0*np.arcsin(np.sqrt(_sum))
    

def pointToLineDistance(lon1, lat1, lon2, lat2, lon3, lat3):
    """All radians
    points 1 and 2 define an arc segment,
    this finds the distance of point 3 to the arc segment. 
    """

    result = lon1*0
    needed = np.ones(result.size, dtype=bool)

    bear12 = bear(lat1, lon1, lat2, lon2)
    bear13 = bear(lat1, lon1, lat3, lon3)
    dis13 = _angularSeparation(lon1, lat1, lon3, lat3)

    # Is relative bearing obtuse?
    diff = np.abs(bear13 - bear12)
    if np.size(diff) == 1:
        if diff > np.pi:
            diff = 2*np.pi - diff
        if diff > (np.pi / 2):
            return dis13
    else:
        solved = np.where(diff > (np.pi / 2))[0]
        result[solved] = dis13[solved]
        needed[solved] = 0
    
    # Find the cross-track distance.
    dxt = np.arcsin(np.sin(dis13) * np.sin(bear13 - bear12))

    # Is p4 beyond the arc?
    dis12 = _angularSeparation(lon1, lat1, lon2, lat2)
    dis14 = np.arccos(np.cos(dis13) / np.cos(dxt))
    if np.size(dis14) == 1:
        if dis14 > dis12:
            return _angularSeparation(lon2, lat2, lon3, lat3)
    else:
        solved = np.where(dis14 > dis12)[0]
        result[solved] = _angularSeparation(lon2[solved], lat2[solved], lon3[solved], lat3[solved])

    if np.size(lon1) == 1:
        return np.abs(dxt)
    else:
        result[needed] = np.abs(dxt[needed])
        return result

In [61]:
from part1 import pointToLineDistance
import numpy as np 
from shapely.geometry import LineString
from shapely.geometry import Point
from rubin_sim.utils import Site
import ephem
from rubin_sim.utils import _angularSeparation, _buildTree, xyz_angular_radius
from rubin_sim.scheduler.utils import read_fields
from astropy import time
import numpy

class Constellation(object):
    """
    Have a class to hold ephem satellite objects

    Parameters
    ----------
    sat_tle_list : list of str
        A list of satellite TLEs to be used
    tstep : float (5)
        The time step to use when computing satellite positions in an exposure
    """

    def __init__(self, sat_tle_list, alt_limit=30., fov=3.5, tstep=5., exptime=30., seed=42):
        np.random.seed(seed)
        self.sat_list = [ephem.readtle(tle.split('\n')[0], tle.split('\n')[1], tle.split('\n')[2]) for tle in sat_tle_list]
        self.alt_limit_rad = np.radians(alt_limit)
        self.fov_rad = np.radians(fov)
        self._make_observer()
        self._make_fields()
        self.tsteps = np.arange(0, exptime+tstep, tstep)/3600./24.  # to days

        self.radius = xyz_angular_radius(fov)

    def _make_fields(self):
        """
        Make tesselation of the sky
        """
        # RA and dec in radians
        fields = read_fields()

        # crop off so we only worry about things that are up
        good = np.where(fields['dec'] > (self.alt_limit_rad - self.fov_rad))[0]
        self.fields = fields[good]

        self.fields_empty = np.zeros(self.fields.size)

        # we'll use a single tessellation of alt az
        leafsize = 100
        self.tree = _buildTree(self.fields['RA'], self.fields['dec'], leafsize, scale=None)

    def _make_observer(self):
        telescope = Site(name='LSST')

        self.observer = ephem.Observer()
        self.observer.lat = telescope.latitude_rad
        self.observer.lon = telescope.longitude_rad
        self.observer.elevation = telescope.height

    def advance_epoch(self, advance=100):
        """
        Advance the epoch of all the satellites
        """

        # Because someone went and put a valueError where there should have been a warning
        # I prodly present the hackiest kludge of all time
        for sat in self.sat_list:
            sat._epoch += advance

    def set_epoch(self, mjd):
        for sat in self.sat_list:
            sat._epoch = mjd

    #self.update_mjd gives a bunch of positions 
    def update_mjd(self, mjd, indx=None):
        """
        mjd : float
            The MJD to advance the satellites to
        indx : list-like of ints
            Only propigate a subset of satellites. 
        """
        self.active_indx = indx

        self.observer.date = ephem.date(time.Time(mjd, format='mjd').datetime)

        self.altitudes_rad = []
        self.azimuth_rad = []
        self.eclip = []
        if self.active_indx is None:
            indx = np.arange(len(self.sat_list))
        else:
            indx = self.active_indx
        for i in indx:
            sat = self.sat_list[i]
            try:
                sat.compute(self.observer)
            except ValueError:
                self.set_epoch(self.observer.date+np.random.uniform()*10)
                sat.compute(self.observer)
            self.altitudes_rad.append(sat.alt)
            self.azimuth_rad.append(sat.az)
            self.eclip.append(sat.eclipsed)

        self.altitudes_rad = np.array(self.altitudes_rad)
        self.azimuth_rad = np.array(self.azimuth_rad)
        self.eclip = np.array(self.eclip)
        # Keep track of the ones that are up and illuminated
        self.above_alt_limit = np.where((self.altitudes_rad >= self.alt_limit_rad) & (self.eclip == False))[0]
    
    ##numpy doc docstring 
    def check_pointing(self, pointing_alt, pointing_az, mjd, exposure_time):
        """Calculates the length of satellite streaks in a pointing. 
        Parameters
        ----------
        Param1 : float 
            the altitude of the pointing
        Param2 : float
            the azimuth of the pointing
        Param3 : float
            the current mjd
        Param4: float 
            the length of exposure.

        Returns
        -------
        list
            list of streak length in the given pointing"""
        pointing_alt=np.radians(pointing_alt)
        pointing_az=np.radians(pointing_az)
        streak_len=[]
        # sat_list=self.sat_list
        self.update_mjd(mjd)
        inLat_list=self.altitudes_rad
        inLong_list=self.azimuth_rad
        initialPositions=list(zip(inLat_list,inLong_list))
        self.update_mjd(mjd+exposure_time)
        finLat_list=self.altitudes_rad
        finLong_list=self.azimuth_rad  
        endPositions=list(zip(finLat_list,finLong_list))
        for i in range(len(initialPositions)):
        # for i in range(40):
            initial_lat, initial_lon=initialPositions[i]
            end_lat, end_lon = endPositions[i]
            distance=pointToLineDistance(initial_lat, initial_lon, end_lat, end_lon, pointing_alt, pointing_az)
            if distance<np.radians(1.75):
                streak=calculate_length(initial_lat, initial_lon, end_lat, end_lon, pointing_alt, pointing_az, np.radians(1.75))
                streak_len.append(streak)
        return streak_len

def calculate_length(initial_lat, initial_lon, end_lat, end_lon, pointing_alt, pointing_az, radius ):
    """Helper funciton for check_pointing. 
    calculate the length of a streak after projecting the locations of the satellite and the pointing onto 2D.
    Parameters
    ----------
    Param1 : float 
        the initial latitude of the satellite
    Param2 : float
        the initial longitude of the satellite
    Param3 : float
        the end latitude of the satellite
    Param4: float 
        the end longitude of the satellite
    Param5 : float
        the altitude of the pointing 
    Param6: float 
        the azimuth of the pointing
    Param7 : float
        the radius of the pointing 


    Returns
    -------
    float
        the length of the satellite streak in the pointing 
    """
    #stsart location 
    x1,y1=gnomonic_project_toxy(initial_lat, initial_lon, pointing_alt, pointing_az)
    #end location
    x2,y2=gnomonic_project_toxy(end_lat, end_lon, pointing_alt, pointing_az)
    #center of pointing 
    x_c,y_c=gnomonic_project_toxy(pointing_alt, pointing_az, pointing_alt, pointing_az)
    #TODO: so is radius just the old radius?? 


    #from https://stackoverflow.com/questions/30844482/what-is-most-efficient-way-to-find-the-intersection-of-a-line-and-a-circle-in-py
    p = Point(x_c,y_c)
    circle = p.buffer(radius).boundary
    # print(circle)
    line = LineString([(x1,y1), (x2,y2)])
    i = circle.intersection(line)
    try:
        p1=i.geoms[0].coords[0]
        p2=i.geoms[1].coords[0]
    except:
        print(p)
        print(radius)
        print(line)
    #fix later 
        pass
    else: 
        len=np.sqrt((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)
        return len 







#need to project start point, end point, center, and radius 
def gnomonic_project_toxy(RA1, Dec1, RAcen, Deccen):
    """Calculate x/y projection of RA1/Dec1 in system with center at RAcen, Deccen.
    Input radians. Grabbed from sims_selfcal
    Parameters
    ----------
    Param1 : float 
        the right ascension of the object
    Param2 : float
        the declination of the object
    Param3 : float
        the right ascension of the center of the system
    Param4: float 
        the declination of the center of the system



    Returns
    -------
    list
        two element list that contains the x,y projection of the object 
    
    
    """
    # also used in Global Telescope Network website
    cosc = np.sin(Deccen) * np.sin(Dec1) + np.cos(Deccen) * np.cos(Dec1) * np.cos(
        RA1 - RAcen
    )
    x = np.cos(Dec1) * np.sin(RA1 - RAcen) / cosc
    y = (
        np.cos(Deccen) * np.sin(Dec1)
        - np.sin(Deccen) * np.cos(Dec1) * np.cos(RA1 - RAcen)
    ) / cosc
    return x, y

In [11]:
tle_list=starlink_constellation()

In [62]:
test_const=Constellation(tle_list)
#df is in degree 

# radius = 1.75 degrees (field of view)
np.radians(1.75)

In [63]:
test_const.check_pointing(345.028793, -48.927820, 63870.106090, 30.0)

POINT (0 0)
0.030543261909900768
LINESTRING (0.9857367381941111 -3.869604892388161, 0.3778131003564072 -1.4079161773002522)
POINT (0 0)
0.030543261909900768
LINESTRING (0.3246624745741397 -1.376876070449937, 1.1995432707189833 -5.157860070935604)
POINT (0 0)
0.030543261909900768
LINESTRING (-0.0422200160857122 -0.8420534087148548, -0.0153901001642175 -0.0640966516468646)
POINT (0 0)
0.030543261909900768
LINESTRING (-0.0072170954162709 -0.0980542153763192, 0.0189512031675949 -0.8880629300811715)
POINT (0 0)
0.030543261909900768
LINESTRING (0.0175912115395981 -0.0010563941742892, -0.0309872728836407 -0.8466807915987472)
POINT (0 0)
0.030543261909900768
LINESTRING (-0.0087107768611678 -0.8648607709820122, -0.386597859129025 -18.373664940445078)
POINT (0 0)
0.030543261909900768
LINESTRING (0.097758321062128 -2.6855648764184963, 0.0228590514799709 -0.8900750380649103)
POINT (0 0)
0.030543261909900768
LINESTRING (-0.1571034432861135 -0.357043041768904, -0.1929302959044634 -0.4551855592184194

[None,
 0.06093222237694784,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.057563792539030274,
 0.050421573764836074,
 0.031057483886026204,
 None,
 None,
 0.047096059767527634,
 0.0400317532520362,
 0.055696398999713397,
 0.04959350254719904,
 0.059680930964542675,
 0.059905094563497456,
 None,
 None,
 0.03851457945521767,
 0.047831752889162076,
 0.028720670974135193,
 0.035821447174418916,
 0.059667774861907316,
 0.056424842986785836,
 0.039511098722688376,
 0.026331661139344372,
 0.03477538255003329,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 0.04965251042789043,
 None,
 0.023864044811970266,
 0.05081807358464092,
 0.021672722229735854,
 None,
 0.06103804940716431,
 None,
 None,
 None,
 0.06007452092241718,
 0.05280733684819843,
 0.045921984107711276,
 0.060250319277161776,
 0.05191332839519294,
 0.04602345073539883,
 0.05856934953622648,
 0.05343509197004193,
 0.0432057172212905,
 None,
 None,
 0.05871006865030479,
 0.026231374